In [1]:
import sys
import os
import re
import datetime
import json

sys.path.append(os.getcwd())

from backend import config
from backend.modelClasses import PromptHandler, LLMWrapper, TeacherAgent
from backend.tts import TTSProcessor
from backend.pipeline import load_scenario, run_pipeline
from backend.utils import reformat_transcript_to_list
from backend.prompts.scenarios import *


/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Scenario Prep

In [2]:
# Read the scenario from a file
scenario_file_path = "./backend/prompts/scenarios/citytour.txt"
with open(scenario_file_path, "r", encoding="utf-8") as file:
    scenario = file.read()

# Set other parameters
country_name = "France"
language = "French"


### Transcript Generation

In [3]:
# Create the prompt handler and transcript generator instances
prompt_handler = PromptHandler(config.SYSTEM_PROMPT_TEMPLATE_PATH)
transcript_generator = LLMWrapper(config.MODEL_NAME, temperature=0.0)

# Format the prompt and generate the transcript using keyword arguments
prompt = prompt_handler.format_prompt(scenario=scenario, country_name=country_name, language=language)
print("----- Prompt -----")
print(prompt)
raw_transcript = transcript_generator.generate(prompt)

print("----- Raw Transcript -----")
print(raw_transcript)


----- Prompt -----
You are an expert content creator for French language learners. Your role is to produce a **natural-sounding, French-only conversation** between two characters, strictly following the formatting rules.

## **Scenario:**  
The conversation must be based on the following scenario: **A tour of the city is about to start.
The main character joins at the tour starting point.
The tour guide introduces themself and starts the tour.
**.

## **Requirements:**
1. **French Only**
   - **Do not include any English text** except for character names.
   - Reflect the **local dialect** and **culture** of France in your French.

2. **Strict Formatting Rules**
   - **Every single line must follow this format, with NO exceptions:**
     ```
     Speaker Name: "Dialogue text."
     ```
   - **Examples of correct formatting:**
     ```
     Ana: "Hola, ¿cómo estás?"
     David: "Estoy muy bien, gracias."
     ```
   - **INCORRECT formatting (DO NOT DO THIS):**
     - `Ana (smiling): "Ho

In [4]:
# Create the teacher agent instance
teacher_agent = TeacherAgent(
    template_path=config.TEACHER_PROMPT_TEMPLATE_PATH,
    model_name=config.MODEL_NAME,
    temperature=0.0,
)

# Generate the explained transcript using keyword arguments for additional context
explained_transcript = teacher_agent.explain(
    raw_transcript, 
    scenario=scenario, 
    country_name=country_name,
    language = language
)

print("----- Explained Transcript -----")
print(explained_transcript)


----- Explained Transcript -----
Guillaume: "Bonjour, Madame! Vous êtes ici pour la visite de la ville?"

Maestro: "Guillaume is asking if the woman is here for the city tour."

Visiteuse: "Oui, c'est ma première fois ici. Je suis très excitée!"

Maestro: "The woman confirms that it's her first time here and expresses her excitement."

Guillaume: "C'est un plaisir de vous accueillir, Madame. Je suis Guillaume, votre guide pour la journée. Alors, suivons-nous, s'il vous plaît."

Maestro: "Guillaume welcomes the woman and introduces himself as her guide for the day."

Visiteuse: "Merci, Guillaume. Je suis Ana."

Maestro: "Ana introduces herself to Guillaume."

Guillaume: "Enchantée, Ana. Nous commençons par la place centrale de notre ville, la Place de la République. Vous verrez les belles architectures et l'énergie de nos citoyens."

Maestro: "Guillaume tells Ana that they will start at the city's central square, Place de la République, where she can see beautiful architecture and the e

In [5]:
unique_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
conversation_dir = os.path.join(config.GENERATED_OUTPUT_DIR, unique_id)
os.makedirs(conversation_dir, exist_ok=True)
final_audio_file = os.path.join(conversation_dir, "final_conversation.wav")


In [6]:
transcript_lines = reformat_transcript_to_list(explained_transcript)
transcript_lines

['Guillaume: "Bonjour, Madame! Vous êtes ici pour la visite de la ville?"',
 'Maestro: "Guillaume is asking if the woman is here for the city tour."',
 'Visiteuse: "Oui, c\'est ma première fois ici. Je suis très excitée!"',
 'Maestro: "The woman confirms that it\'s her first time here and expresses her excitement."',
 'Guillaume: "C\'est un plaisir de vous accueillir, Madame. Je suis Guillaume, votre guide pour la journée. Alors, suivons-nous, s\'il vous plaît."',
 'Maestro: "Guillaume welcomes the woman and introduces himself as her guide for the day."',
 'Visiteuse: "Merci, Guillaume. Je suis Ana."',
 'Maestro: "Ana introduces herself to Guillaume."',
 'Guillaume: "Enchantée, Ana. Nous commençons par la place centrale de notre ville, la Place de la République. Vous verrez les belles architectures et l\'énergie de nos citoyens."',
 'Maestro: "Guillaume tells Ana that they will start at the city\'s central square, Place de la République, where she can see beautiful architecture and the

In [7]:
tts_processor = TTSProcessor(api_key=config.ELEVEN_API_KEY)

tts_processor.process_transcript(
    transcript_lines=transcript_lines,
    output_dir=conversation_dir,
    final_audio_file=final_audio_file,
    language=language,
    country=country_name
    
)

print("Audio generated for each line, and combined audio saved at:")
print(final_audio_file)

Inferred speaker genders: {'Guillaume': 'male', 'Visiteuse': 'female', 'Ana': 'female'}
Synthesizing line 1 for 'Guillaume' using voice 'hv6gVog5LgtIUX88Nmq8'...
Synthesizing line 2 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 3 for 'Visiteuse' using voice 'pVsdIxxCbRrbiwOhiRwg'...
Synthesizing line 4 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 5 for 'Guillaume' using voice 'hv6gVog5LgtIUX88Nmq8'...
Synthesizing line 6 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 7 for 'Visiteuse' using voice 'pVsdIxxCbRrbiwOhiRwg'...
Synthesizing line 8 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 9 for 'Guillaume' using voice 'hv6gVog5LgtIUX88Nmq8'...
Synthesizing line 10 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 11 for 'Ana' using voice 'pVsdIxxCbRrbiwOhiRwg'...
Synthesizing line 12 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 13 for 'Guillaume' us